In [ ]:
# !pip install scikit-learn

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
from sklearn import preprocessing

df = pd.read_csv('nri-svi-cleaned.csv')

df_numerical = df[['EP_POV150','EP_HBURD','EP_AFAM','EP_HISP','RISK_VALUE','CFLD_RISKV','HWAV_RISKV']]


f = lambda x: np.power(abs(x), 1./3)
df_numerical['RISK_VALUE'] = f(df_numerical['RISK_VALUE'])
# df_numerical['EAL_VALT'] = f(df_numerical['EAL_VALT'])


df_numerical['CFLD_RISKV'] = f(df_numerical['CFLD_RISKV'])
df_numerical['HWAV_RISKV'] = f(df_numerical['HWAV_RISKV'])

# print(df_numerical)
df_numerical=(df_numerical-df_numerical.min())/(df_numerical.max()-df_numerical.min())
# df_numerical['RISK_VALUE'] = df_numerical['RISK_VALUE']*4
# df_numerical.to_csv('nri-svi-cleaned-2.csv', sep=',', index=False, encoding='utf-8')

In [ ]:
silhouette_scores = []
# Determine the number of clusters using Silhouette Score
for num_clusters in range(2, 11):
    kmeans = KMeans(n_clusters=num_clusters, init='k-means++', max_iter=300, n_init=10, random_state=0)
    cluster_labels = kmeans.fit_predict(df_numerical.iloc[:, :-1])
    silhouette_avg = silhouette_score(df_numerical.iloc[:, :-1], cluster_labels)
    silhouette_scores.append(silhouette_avg)


# Plotting the Silhouette Score graph
plt.figure(figsize=(8, 6))
plt.plot(range(2, 11), silhouette_scores, marker='o', linestyle='--')
plt.title('Silhouette Score Method')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Score')
plt.show()

# Find the number of clusters with the highest Silhouette Score
best_num_clusters = silhouette_scores.index(max(silhouette_scores)) + 2

# Perform K-Means clustering with the best number of clusters, in this case k=3
kmeans = KMeans(n_clusters=best_num_clusters, init='k-means++', max_iter=300, n_init=10, random_state=0)
cluster_labels = kmeans.fit_predict(df_numerical.iloc[:, :-1])

# Add the cluster labels to the DataFrame
df_numerical['cluster'] = cluster_labels
centroids  = kmeans.cluster_centers_
# print("k=3")
# print(centroids)

# # Perform K-Means clustering with k=2
# kmeans = KMeans(n_clusters=2, init='k-means++', max_iter=300, n_init=10, random_state=0)
# cluster_labels = kmeans.fit_predict(df_numerical.iloc[:, :-1])

# centroids  = kmeans.cluster_centers_
# print("k=2")
# print(centroids)

# # Perform K-Means clustering with k=5
# kmeans = KMeans(n_clusters=4, init='k-means++', max_iter=300, n_init=10, random_state=0)
# cluster_labels = kmeans.fit_predict(df_numerical.iloc[:, :-1])

# centroids  = kmeans.cluster_centers_
# print("k=4")
# print(centroids)


# print(df_numerical.head(5))
# print(f"The best number of clusters according to Silhouette Score is: {best_num_clusters}")

In [ ]:
# save data with cluster labels and FIPS labels
df_numerical.insert(0,'FIPS', df['FIPS'])
df_numerical.to_csv('kmeans-groups.csv', sep=',', index=False, encoding='utf-8')

In [ ]:
#Importing required modules
from sklearn.decomposition import PCA
import numpy as np
 
pca = PCA(n_components=2)
 
#Transform the data
df = pca.fit_transform(df_numerical.iloc[:,:-1])
 
#Getting unique labels
u_labels = np.unique(cluster_labels)
colors_3 = ['blue','orange','green','red']

#plotting the results:
for i in u_labels:
    plt.scatter(df[cluster_labels == i , 0] , df[cluster_labels == i , 1] , s = 10, label = f"cluster {i}")
    plt.scatter(centroids[i,0] , centroids[i,1] , s = 40, color = colors_3[i], label = f"centroid {i}")

plt.title("K-means cluster plot with k=3 (\"best\" k), using PCA")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.legend()
plt.show()

print(pca.components_)
# print(['EP_POV150','EP_HBURD','EP_AFAM','EP_HISP','RISK_VALUE','CFLD_RISKV','HWAV_RISKV'])